In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm

In [2]:
# Upload all datasets
word_categorization = pd.read_csv("word_categorization.csv")
word_recognition = pd.read_csv("word_recognition.csv")
word_frequency = pd.read_csv("word_frequency_tot_categorization_task.csv")
wc_mt_statistics = pd.read_csv("WC_MT_statistics.csv", header=[0])

In [3]:
df_user_10106 = word_categorization[word_categorization['user_id'] == 10106]
#df_user_10106

# VCheck for non-null values in column ‘response_time_ms’ for user 10106
non_nan_response_times = df_user_10106['response_time_ms'].dropna()

# Count how many non-zero values there are
num_non_nan_values = non_nan_response_times.count()

num_non_nan_values

0

In [4]:
word_categorization.head()

,user_id,voucher,valid,language,word,user_given_response,expected_correct_response,is_user_response_correct,alive_notAlive,manmade_notManmade,response_time_ms
0,10106,1,1,0,HIGHWAY,0,0,1,0.0,NaN,NaN
1,10106,1,1,0,CLOVER,0,1,0,1.0,NaN,NaN
2,10106,1,1,0,KING,1,1,1,1.0,NaN,NaN
3,10106,1,1,0,NAPKIN,0,0,1,0.0,NaN,NaN
4,10106,1,1,0,BARREL,0,0,1,0.0,NaN,NaN


In [5]:
wc_mt_statistics.head() # 51 to go beyond null

,user_id,voucher,valid,language,WC_CorrAns_RT_mean_1,WC_CorrAns_RT_SD_1,WC_correct_answers_percentage_1,WC_information_1,WC_information_rate_1,MT_IC_1,MT_EC_1,MT_IW_1,MT_EW_1,MT_NG_1,MT_C_1
0,10106,1,1,0,NaN,NaN,83.076923,0.344058,NaN,14.0,31.5,11.5,19.0,24.0,45.5
1,10162,1,1,0,NaN,NaN,93.846154,0.666478,NaN,55.0,1.0,25.5,2.5,16.0,56.0
2,100,1,1,0,NaN,NaN,96.923077,0.801765,NaN,14.5,65.0,10.0,4.0,6.5,79.5
3,10154,1,1,0,NaN,NaN,92.307692,0.608756,NaN,0.0,64.0,0.5,34.0,1.5,64.0
4,10144,1,1,0,NaN,NaN,94.615385,0.697484,NaN,19.0,40.5,11.0,6.5,23.0,59.5


In [6]:
# Check for the presence of NaN
nan_summary = wc_mt_statistics.isnull().sum()
nan_columns = nan_summary[nan_summary > 0]

# Show columns with NaN values and their amount
print("Columns with NaN values and their amount:")
print(nan_columns)

Columns with NaN values and their amount:
WC_CorrAns_RT_mean_1     49
WC_CorrAns_RT_SD_1       49
WC_information_rate_1    49
dtype: int64


In [7]:
# doing the median for the null values (inplace =True)
# train_data.Mri_Track_age_scan.fillna(train_data.Mri_track_age_scan.median(), inplace=True)

In [8]:
# Count the number of unique user_id's in the file
unique_user_ids = word_categorization['user_id'].nunique()
unique_user_ids

235

In [11]:
# Create df_comparison_filtered by removing records with NaN in key columns
columns_to_filter = ["WC_CorrAns_RT_mean_1", "WC_CorrAns_RT_SD_1", "WC_information_rate_1"]
df_comparison_filtered = wc_mt_statistics.dropna(subset=columns_to_filter).copy().reset_index()

# Recalculate metrics based on filtered data
df_comparison_filtered["calculated_WC_CorrAns_RT_mean_1"] = df_comparison_filtered.groupby("user_id")["WC_CorrAns_RT_mean_1"].transform("mean")
df_comparison_filtered["calculated_WC_CorrAns_RT_SD_1"] = df_comparison_filtered.groupby("user_id")["WC_CorrAns_RT_SD_1"].transform("std")
df_comparison_filtered["calculated_WC_correct_answers_percentage_1"] = df_comparison_filtered.groupby("user_id")["WC_information_rate_1"].transform("mean")

In [14]:
print(df_comparison_filtered.columns)

Index(['index', 'user_id', 'voucher', 'valid', 'language',
       'WC_CorrAns_RT_mean_1', 'WC_CorrAns_RT_SD_1',
       'WC_correct_answers_percentage_1', 'WC_information_1',
       'WC_information_rate_1', 'MT_IC_1', 'MT_EC_1', 'MT_IW_1', 'MT_EW_1',
       'MT_NG_1', 'MT_C_1', 'calculated_WC_CorrAns_RT_mean_1',
       'calculated_WC_CorrAns_RT_SD_1',
       'calculated_WC_correct_answers_percentage_1', 'mean_match', 'std_match',
       'accuracy_match'],
      dtype='object')


### VERIFICATION OF REACTION TIMES

In [12]:
# Round values to 3 decimal places before comparison
df_comparison_filtered["WC_CorrAns_RT_mean_1"] = df_comparison_filtered["WC_CorrAns_RT_mean_1"].round(3)
df_comparison_filtered["calculated_WC_CorrAns_RT_mean_1"] = df_comparison_filtered["calculated_WC_CorrAns_RT_mean_1"].round(3)

df_comparison_filtered["WC_CorrAns_RT_SD_1"] = df_comparison_filtered["WC_CorrAns_RT_SD_1"].round(3)
df_comparison_filtered["calculated_WC_CorrAns_RT_SD_1"] = df_comparison_filtered["calculated_WC_CorrAns_RT_SD_1"].round(3)

df_comparison_filtered["WC_information_rate_1"] = df_comparison_filtered["WC_information_rate_1"].round(3)
df_comparison_filtered["calculated_WC_correct_answers_percentage_1"] = df_comparison_filtered["calculated_WC_correct_answers_percentage_1"].round(3)

# Compare if values match to 3 decimals
df_comparison_filtered["mean_match"] = df_comparison_filtered["WC_CorrAns_RT_mean_1"] == df_comparison_filtered["calculated_WC_CorrAns_RT_mean_1"]
df_comparison_filtered["std_match"] = df_comparison_filtered["WC_CorrAns_RT_SD_1"] == df_comparison_filtered["calculated_WC_CorrAns_RT_SD_1"]
df_comparison_filtered["accuracy_match"] = df_comparison_filtered["WC_information_rate_1"] == df_comparison_filtered["calculated_WC_correct_answers_percentage_1"]

# Coincidences and discrepancies
mean_match_count_filtered = df_comparison_filtered["mean_match"].sum()
std_match_count_filtered = df_comparison_filtered["std_match"].sum()
accuracy_match_count_filtered = df_comparison_filtered["accuracy_match"].sum()

total_users_filtered = df_comparison_filtered.shape[0]

{
    "Total users (without NaN values)": total_users_filtered,
    "Mean reaction time matches (3 decimals)": mean_match_count_filtered,
    "STD reaction time matches (3 decimals)": std_match_count_filtered,
    "Accuracy matches (3 decimals)": accuracy_match_count_filtered
}


{'Total users (without NaN values)': 186,
 'Mean reaction time matches (3 decimals)': 186,
 'STD reaction time matches (3 decimals)': 0,
 'Accuracy matches (3 decimals)': 186}

In [32]:
# NOTE: it is compared with 3 decimals because few results matched with the default ones but using 3 decimals
# all the registers coincide except for the NaN which are not calculated.

In [ ]:
# NOTE: The problem was due to the fact that the original calculation in the database used the population standard deviation (ddof=0), while the standard method in Pandas (ddof=1) calculates the sample standard deviation. 
#(ddof=0), while the standard method in Pandas (ddof=1) calculates the sample standard deviation, 
#, which generated small differences.

In [15]:
# Calculate the standard deviation with ddof=0 (population) and ddof=1 (sample)
"""user_10150_calculated_sd_population = user_10150_reaction_times.std(ddof=0)
user_10150_calculated_sd_sample = user_10150_reaction_times.std(ddof=1)

# Show results to see if they match
{
    "Stored WC_CorrAns_RT_SD_1": user_10150_stored_value[0] if len(user_10150_stored_value) > 0 else None,
    "Calculated WC_CorrAns_RT_SD_1 (Population)": user_10150_calculated_sd_population,
    "Calculated WC_CorrAns_RT_SD_1 (Sample)": user_10150_calculated_sd_sample
}"""

'user_10150_calculated_sd_population = user_10150_reaction_times.std(ddof=0)\nuser_10150_calculated_sd_sample = user_10150_reaction_times.std(ddof=1)\n\n# Show results to see if they match\n{\n    "Stored WC_CorrAns_RT_SD_1": user_10150_stored_value[0] if len(user_10150_stored_value) > 0 else None,\n    "Calculated WC_CorrAns_RT_SD_1 (Population)": user_10150_calculated_sd_population,\n    "Calculated WC_CorrAns_RT_SD_1 (Sample)": user_10150_calculated_sd_sample\n}'

In [54]:
user_10150_calculated_sd_sample = user_10150_reaction_times.std(ddof=1)  # Sample (n-1)
user_10150_calculated_sd_population = user_10150_reaction_times.std(ddof=0)  # Population (n)
user_10150_calculated_sd_sample, user_10150_calculated_sd_population

(764.2103955403774, 761.1226641432567)

In [71]:
df_comparison_filtered.columns

Index(['user_id', 'voucher', 'valid', 'language', 'WC_CorrAns_RT_mean_1',
       'WC_CorrAns_RT_SD_1', 'WC_correct_answers_percentage_1',
       'WC_information_1', 'WC_information_rate_1', 'MT_IC_1', 'MT_EC_1',
       'MT_IW_1', 'MT_EW_1', 'MT_NG_1', 'MT_C_1',
       'calculated_WC_CorrAns_RT_mean_1_x', 'calculated_WC_CorrAns_RT_SD_1_x',
       'calculated_WC_correct_answers_percentage_1', 'mean_match', 'std_match',
       'accuracy_match', 'calculated_WC_CorrAns_RT_mean_1_y',
       'calculated_WC_CorrAns_RT_SD_1_y'],
      dtype='object')

## FIX the NaN values

Median

In [65]:
# Select the columns to be filled in
columns_to_fill = ["WC_CorrAns_RT_mean_1", "WC_CorrAns_RT_SD_1", "WC_information_rate_1"]

# Fill in the NaN values with the mean of each column
wc_mt_statistics[columns_to_fill] = wc_mt_statistics[columns_to_fill].fillna(wc_mt_statistics[columns_to_fill].mean())

# Check if NaN values are left after filling
nan_after_filling = wc_mt_statistics[columns_to_fill].isnull().sum()

# Show the results
nan_after_filling

WC_CorrAns_RT_mean_1     0
WC_CorrAns_RT_SD_1       0
WC_information_rate_1    0
dtype: int64

In [66]:
wc_mt_statistics

,user_id,voucher,valid,language,WC_CorrAns_RT_mean_1,WC_CorrAns_RT_SD_1,WC_correct_answers_percentage_1,WC_information_1,WC_information_rate_1,MT_IC_1,MT_EC_1,MT_IW_1,MT_EW_1,MT_NG_1,MT_C_1
0,10106,1,1,0,1382.466167,933.743584,83.076923,0.344058,0.594663,14.0,31.5,11.5,19.0,24.0,45.5
1,10162,1,1,0,1382.466167,933.743584,93.846154,0.666478,0.594663,55.0,1.0,25.5,2.5,16.0,56.0
2,100,1,1,0,1382.466167,933.743584,96.923077,0.801765,0.594663,14.5,65.0,10.0,4.0,6.5,79.5
3,10154,1,1,0,1382.466167,933.743584,92.307692,0.608756,0.594663,0.0,64.0,0.5,34.0,1.5,64.0
4,10144,1,1,0,1382.466167,933.743584,94.615385,0.697484,0.594663,19.0,40.5,11.0,6.5,23.0,59.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,10434,1,1,0,1066.629921,593.299725,97.692308,0.841616,0.793229,11.0,54.5,1.5,13.0,20.0,65.5
231,10435,1,1,0,1902.201550,2397.098946,99.230769,0.934927,0.493151,12.5,75.5,5.5,2.5,4.0,88.0
232,10436,1,1,0,1006.580645,597.979880,95.384615,0.730172,0.717744,3.0,62.0,2.0,33.0,0.0,65.0
233,10437,1,1,0,1756.441860,803.851146,99.230769,0.934927,0.525558,62.0,10.5,16.5,0.5,10.5,72.5
